In [1]:
from pydantic import BaseModel, Field
from typing import List, Tuple
import dspy
import sys
import os
sys.path.append(r"D:\CursorProj\Chem-Ontology-Constructor")
os.environ["PROJECT_ROOT"] = "D:\\CursorProj\\Chem-Ontology-Constructor\\"


from autonogy_constructor.modules import ChemOntology, Assessment, ontology_to_string
from autonogy_constructor.base_data_structures import Ontology
from autonogy_constructor.metrics import metric



d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from config.settings import ASSESSMENT_CRITERIA_CONFIG

print(ASSESSMENT_CRITERIA_CONFIG["full_score"])


54


In [3]:

lm = dspy.LM('openai/gpt-4o', temperature=0)
dspy.configure(lm=lm)
teacher = dspy.LM("openai/gpt-4o",temperature=1)


In [4]:
import json
import os

def load_json_files(folder_path, template_str):
    combined_data = []
    
    # 遍历文件夹中的所有文件
    for filename in os.listdir(folder_path):
        # 检查文件是否是json且名称包含模板字符串
        if filename.endswith('.json') and template_str in filename:
            file_path = os.path.join(folder_path, filename)
            # 读取json文件
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                combined_data.extend(data)
                
    return combined_data


In [5]:

# 使用示例
folder_path = r'./data'
template_str = 'content_list'
data = load_json_files(folder_path, template_str)
len(data)


6924

In [6]:
test_context = """
###### How electron donation and withdrawal change chemical shifts  \nWe can get an idea of the effect of electron distribution by looking at a series of benzene rings\nwith the same substituent in the 1 and 4 positions. This pattern makes all four hydrogens on\nthe ring identical. Here are a few compounds listed in order of chemical shift: largest shift\n(lowest fi eld; most deshielded) fi rst. Conjugation is shown by the usual curly arrows, and\ninductive effects by a straight arrow by the side of the group. Only one hydrogen atom and\none set of arrows are shown.  \nConjugation, as discussed in\nChapter 7, is felt through π bonds,\nwhile inductive effects are the\nresult of electron withdrawal or\ndonation felt simply by polarization\nof the σ bonds of the molecule.\nSee p. 135.  \nthe effect of electron-withdrawing groups\nby conjugation  \nby inductive effects  \n**H**  \n**O**  \n**O**  \n**HO**  \n**N**  \nδH 8.48 δH 8.10 **C** δH 8.10 δH 8.07 δH 7.78  \n**N**  \n**O**  \n**O**  \n**OH**  \n**C**  \n**N**  \n**O**  \n**H**  \n**F** **F**  \n**F**  \nThe largest shifts come from groups that withdraw electrons by conjugation. Nitro is the\nmost powerful—this should not surprise you as we saw the same in non-aromatic compounds\nin both [13]C and [1]H NMR spectra. Then come the carbonyl and nitrile group followed by groups\nshowing simple inductive withdrawal. CF3 is an important example of this kind of group—\nthree fl uorine atoms combine to exert a powerful effect.  \n-----  \nIn the middle of our sequence, around the position of benzene itself at 7.27 ppm, come\nthe halogens, whose inductive electron withdrawal and lone pair donation are nearly\nbalanced.  \nbalance between withdrawal by inductive effect and donation of lone pairs by conjugation  \n**I** δH 7.40 **Br** δH 7.32 δH 7.27 **Cl** δH 7.24 **F** δH 7.00  \n**I**  \n**Br**  \n**Cl**  \n**F**  \nAlkyl groups are weak inductive donators, but the groups which give the most shielding—\nperhaps surprisingly—are those containing the electronegative atoms O and N. Despite being\ninductively electron withdrawing (the C–O and C–N σ bonds are polarized with δ + C), on\nbalance conjugation of their lone pairs with the ring (as you saw on p. 278) makes them net\nelectron donors. They increase the shielding at the ring hydrogens. Amino groups are the best.\nNote that one nitrogen-based functional group (NO2) is the best electron withdrawer while\nanother (NH2) is the best electron donor.  \nthe effect of electron-donating groups  \nby inductive effect  \nbalance between withdrawal by inductive effect and donation\nof lone pairs by conjugation—electron donation wins  \n**H**  \nδH 7.03  \n**H**  \n**H**  \n**CH3**  \nδH 6.80 **O**  \n**H** **H**  \nδH 6.59 **N**  \n**H**  \nδH 6.35  \n**H**  \n**H**  \n**CH3**  \n**O**  \n**CH3**  \n**H**  \n**H**  \n**N**  \n**O**  \nδH 7.27  \n**H**  \n**H**  \nδH 7.27  \nδH 5.68  \n**H**  \n**H**  \nδH 5.68  \n**O**  \nδH 6.0  \n**H**  \n**H**  \nδH 7.0  \nδH 4.65  \n**H**  \n**H**  \nδH 6.35  \nAs far as the donors with lone pairs are concerned (the halogens plus O and N), two factors\nare important—the size of the lone pairs and the electronegativity of the element. If we look\nat the four halides at the top of this page the lone pairs are in 2p (F), 3p (Cl), 4p (Br), and 5p (I)\norbitals. In all cases the orbitals on the benzene ring are 2p so the fl uorine orbital is of the\nright size to interact well and the others too large. Even though fl uorine is the most electronegative, it is still the best donor. The others don’t pull so much electron density away, but\nthey can’t give so much back either.\nIf we compare the fi rst row of the p block elements—F, OH, and NH2—all have lone pairs\nin 2p orbitals so now electronegativity is the only variable. As you would expect, the most\nelectronegative element, F, is now the weakest donor.
"""





In [7]:
import random

# 设置开发集和训练集的大小
dev_size = 3
train_size = 10
chunk_size = 300
max_attempts = 5000

# 创建两个列表分别存储开发集和训练集数据
dev_data = []
train_data = []
used_texts = set()  # 用于检查重复

attempts = 0

# 先采样开发集
while len(dev_data) < dev_size and attempts < max_attempts:
    item = random.choice(data)
    if item.get('type') == 'text':
        text = item['text']
        if len(text) > chunk_size and text not in used_texts:
            dev_data.append(item)
            used_texts.add(text)
    attempts += 1

# 再采样训练集
while len(train_data) < train_size and attempts < max_attempts:
    item = random.choice(data)
    if item.get('type') == 'text':
        text = item['text']
        if len(text) > chunk_size and text not in used_texts:
            train_data.append(item)
            used_texts.add(text)
    attempts += 1

print(f"随机抽取到{len(dev_data)}个开发集文本:")
for i, item in enumerate(dev_data, 1):
    print(f"\n{i}. 文本长度: {len(item['text'])}")
    print(f"文本内容: {item['text'][:100]}...")

print(f"\n随机抽取到{len(train_data)}个训练集文本:")
for i, item in enumerate(train_data, 1):
    print(f"\n{i}. 文本长度: {len(item['text'])}")
    print(f"文本内容: {item['text'][:100]}...")



随机抽取到3个开发集文本:

1. 文本长度: 343
文本内容: · The majority of $\alpha$ particles penetrated the foil undeflected. · Some $\alpha$ particles expe...

2. 文本长度: 466
文本内容:  34. Explain why these reactions cannot occur as written. (a) (b) $\begin{array}{r}{\mathrm{Fe^{3+}(...

3. 文本长度: 453
文本内容: The electron configuration of an atom is a designation of how electrons are distributed among variou...

随机抽取到10个训练集文本:

1. 文本长度: 653
文本内容: Up to this point, we have considered internal energy and enthalpy changes for a system in which spec...

2. 文本长度: 606
文本内容: According to VSEPR theory, in both structures (I) and (II) the electron-group geometry around $\math...

3. 文本长度: 322
文本内容: 13. Sulfur forms two compounds with oxygen. In the first  compound, $1.000\mathrm{~g~}$ sulfur is co...

4. 文本长度: 565
文本内容: (a) Show that these data are consistent with the law of multiple proportions. formulas for compounds...

5. 文本长度: 565
文本内容: When the ionization energies of a series of isoelectronic atoms and ions are compa

In [64]:
extractor = ChemOntology()
extractors = [extractor.load(location) for location in ['v1', 'v2', 'medium_100trainset_v1']]


In [100]:
temp = ChemOntology()
temp1 = ChemOntology()
temp2 = ChemOntology()

temp.load('v1.json')
temp1.load('v2')
temp2.load('medium_100trainset_v1')

extractors = []

extractors.append(temp)
extractors.append(temp1)
extractors.append(temp2)

In [101]:
extractors

[extractor = Predict(StringSignature(text -> reasoning, ontology
     instructions='Analyze the provided text from research papers in the field of chemistry to identify all chemistry-related entities, data properties, and object properties within an ontological framework.\n\nFollow these Step-by-Step Analysis:\n\n1. Extract Chemistry-Related Entities:\n  - Identify all significant nouns, proper nouns, and technical terminologies that represent chemistry-related concepts, such as molecules, reactions, compounds, processes, or any substantial entities.\n  - Ensure that you capture entities across different levels of detail, from broad chemical categories to specific molecular structures, to create a comprehensive representation of the subject matter.\n  - Choose names for entities that are specific enough to indicate their meaning without additional context, avoiding overly generic terms.\n  - Consolidate similar entities to avoid redundancy, ensuring each represents a distinct concept a

In [104]:
# 创建devset
devset = []

for sample in dev_data:
    context = sample['text']
    ontology = random.choice(extractors)(context)
    score = 0

    # 添加到devset
    devset.append(dspy.Example(
        context=context,
        ontology=ontology,
        score=score
    ).with_inputs("context", "ontology"))
    print("生成了一个样本")

print(f"生成了{len(devset)}个开发集样本")


生成了10个开发集样本


In [8]:
# 创建devset_no_ontology
devset_no_ontology = []

for sample in dev_data:
    context = sample['text']

    # 添加到devset
    devset_no_ontology.append(dspy.Example(
        context=context
    ).with_inputs("context"))

print(f"生成了{len(devset_no_ontology)}个开发集样本")


生成了3个开发集样本


In [109]:
# 创建trainset
trainset = []

for sample in train_data:
    context = sample['text']
    ontology = random.choice(extractors)(context)
    score = 0
    
    # 添加到trainset
    trainset.append(dspy.Example(
        context=context,
        ontology=ontology,
        score=score
    ).with_inputs("context", "ontology"))
    print("生成了一个样本")

print(f"生成了{len(trainset)}个训练集样本")


生成了一个样本
生成了一个样本
生成了一个样本
生成了一个样本
生成了一个样本
生成了一个样本


<unknown>:27: SyntaxWarning: invalid escape sequence '\D'


生成了一个样本
生成了一个样本
生成了一个样本
生成了一个样本
生成了10个训练集样本


In [9]:
# 创建trainset_no_ontology
trainset_no_ontology = []

for sample in train_data:
    context = sample['text']
    
    # 添加到trainset_no_ontology
    trainset_no_ontology.append(dspy.Example(
        context=context
    ).with_inputs("context"))

print(f"生成了{len(trainset_no_ontology)}个训练集样本")


生成了10个训练集样本


In [113]:
# 将trainset和devset保存为json格式
import json

# 将数据集转换为可序列化的格式
def convert_to_serializable(dataset):
    serializable_data = []
    for example in dataset:
        serializable_data.append({
            'context': example.context,
            'ontology': ontology_to_string(example.ontology.ontology),
            'score': example.score
        })
    return serializable_data


In [114]:

# 保存trainset
with open('data/trainset.json', 'w', encoding='utf-8') as f:
    json.dump(convert_to_serializable(trainset), f, ensure_ascii=False, indent=2)

# 保存devset
with open('data/devset.json', 'w', encoding='utf-8') as f:
    json.dump(convert_to_serializable(devset), f, ensure_ascii=False, indent=2)

print("数据集已保存为JSON格式到data目录")


数据集已保存为JSON格式到data目录


In [ ]:

# 从JSON文件读取数据集
def load_datasets_from_json():
    # 读取trainset
    with open('data/trainset.json', 'r', encoding='utf-8') as f:
        trainset_data = json.load(f)
    
    # 读取devset  
    with open('data/devset.json', 'r', encoding='utf-8') as f:
        devset_data = json.load(f)
        
    # 转换回dspy.Example格式
    trainset = []
    devset = []
    
    for item in trainset_data:
        example = dspy.Example(
            context=item['context'],
            ontology=Ontology(item['ontology']),
            score=item['score']
        ).with_inputs("context", "ontology")
        trainset.append(example)
        
    for item in devset_data:
        example = dspy.Example(
            context=item['context'], 
            ontology=Ontology(item['ontology']),
            score=item['score']
        ).with_inputs("context", "ontology")
        devset.append(example)
        
    return trainset, devset

# 加载数据集
trainset, devset = load_datasets_from_json()
print("已从JSON文件加载数据集")


In [13]:
print(devset_no_ontology[0])

Example({'context': '97.A $7.26\\,\\mathrm{kg}$ shot (as used in the sporting event, the shot put) is dropped from the top of a building $168~\\mathrm{m}$ high. What is the maximum temperature increase that could occur in the shot? Assume a specific heat capacity of $0.47\\,\\mathrm{J}\\,\\mathrm{g}^{-1}\\,^{\\circ}\\mathrm{C}^{-1}$ for the shot. Why would the actual measured temperature increase likely be less than the calculated value?\n\n '}) (input_keys={'context'})


In [108]:
# 提取并格式化输出devset的context和ontology
for dev in devset:
    print("Context:")
    print(dev.context)
    print("\nOntology:")
    print(ontology_to_string(dev.ontology.ontology))
    print("\n" + "-"*50 + "\n")


Context:
The situation with magnesium chloride is similar. In magnesium chloride, found in trace quantities in table salt, magnesium atoms lose two electrons to become magnesium ions, $\mathrm{Mg}^{2+}$ $\mathrm{Mg}$ is in group 2). To obtain an electrically neutral formula unit, there must be two $\mathrm{Cl}^{-}$ ions, each with a charge of $^{-1}$ for every $\mathrm{Mg}^{2+}$ ion. The formula of magnesium chloride is $\mathrm{MgCl}_{2}$ 

Ontology:
Entities:
  - Name: Magnesium Chloride(MgCl2)
    Information: Magnesium chloride is a compound found in trace quantities in table salt, with the formula MgCl2.
  - Name: Magnesium Ion(Mg2+)
    Information: Magnesium ions are formed when magnesium atoms lose two electrons, resulting in a charge of +2.
  - Name: Chloride Ion(Cl-)
    Information: Chloride ions have a charge of -1 and pair with magnesium ions to form magnesium chloride.

Data Properties:
  - Name: charge_of_magnesium_ion
    Information: The charge of the magnesium ion is 

In [86]:
scores = []
extractor = ChemOntology()
for dev in devset[:1]:
    pred = extractor(dev['context'])
    score = metric(dev, pred, verbose=True)
    print("完成一个样本")
    scores.append(score)


完成一个样本


In [87]:

for score in scores:
    print(score)
    print("-------------------------------------------------")





Entity Accuracy Score: 5
Data Property Correctness Score: 4
Object Property Completeness Score: 3
Ontology Structure Score: 3
Overall Score: 8

Total Score: 23
Percentage Score: 85.19%

-------------------------------------------------


In [10]:
from dspy.evaluate import Evaluate

evaluate = Evaluate(devset=devset_no_ontology, num_threads=8, display_progress=True, display_table=10)

In [11]:
import dspy.evaluate


evaluate(ChemOntology(), devset=devset_no_ontology, metric=metric)


Average Metric: 4.66 / 10 (46.6%): 100%|██████████| 10/10 [01:54<00:00, 11.41s/it]

2024/11/23 17:37:10 INFO dspy.evaluate.evaluate: Average Metric: 4.663095238095238 / 10 (46.6%)


,context,reasoning,ontology,metric
0,e begin this chapter with a brief survey of early chemical discov-...,The text provides an overview of fundamental concepts in chemistry...,"entities=[Entity(name=""Dalton's Atomic Theory"", information=""Dalto...",✔️ [0.696]
1,"Had we rounded $4.07\:\mathrm{g\,S\times(1\,mol\,S/32.06\,\mathrm{...",The text provided is a mathematical calculation related to chemist...,"entities=[Entity(name='Sulfur(S)', information='Sulfur is a chemic...",✔️ [0.610]
2,"For most groups, the atom of the second row has a lower electron a...",The text discusses the concept of electron affinity in the context...,"entities=[Entity(name='Second Row Atoms', information='Second row ...",✔️ [0.473]
3,The electronic configuration of ${\mathrm{O}_{2}}^{+}$ is $\sigma_...,The text describes the electronic configuration of the diatomic mo...,"entities=[Entity(name='O2_plus', information='${\\mathrm{O}_{2}}^{...",✔️ [0.548]
4,"With a minimum of calculation, estimate the final temperature reac...",The text describes a scenario involving the mixing of two volumes ...,"entities=[Entity(name='WaterSample1', information='WaterSample1 is...",✔️ [0.619]
5,"Thus, $12\;\mathrm{mol}\;\mathrm{CO}_{2}$ are produced for every 2...",The text describes a chemical reaction where a specific amount of ...,"entities=[Entity(name='CarbonDioxide(CO2)', information='Carbon di...",✔️ [0.357]
6,The electron configuration of this excited state suggests a molecu...,The text discusses the electron configuration and bonding in a mol...,"entities=[Entity(name='Methane(CH4)', information='Methane is a mo...",✔️ [0.608]
7,density in that bond. If there are bonds in the molecule that have...,The text discusses the electron density in bonds within a molecule...,"entities=[Entity(name='SulfurAtom', information='A sulfur atom is ...",✔️ [0.383]
8,A $1.250~\mathrm{g}$ sample of meat is heated with concen- trated ...,NaN,NaN,
9,140. Consider the chemical equation below. What is the maximum num...,The text presents a chemical reaction problem involving the calcul...,"entities=[Entity(name='Potassium Permanganate (KMnO4)', informatio...",✔️ [0.369]


46.63

In [46]:
# Import the optimizer
from dspy.teleprompt import MIPROv2

# Initialize optimizer
teleprompter = MIPROv2(
    metric=metric,
    auto="medium",
    teacher_settings=dict(lm=teacher)
)





In [47]:


zeroshot_optimized_program = teleprompter.compile(
    ChemOntology(),
    trainset=trainset,
    max_bootstrapped_demos=0, # ZERO FEW-SHOT EXAMPLES
    max_labeled_demos=0, # ZERO FEW-SHOT EXAMPLES
    requires_permission_to_run=False,
)


2024/11/22 15:35:23 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 25
valset size: 80

2024/11/22 15:35:23 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2024/11/22 15:35:23 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2024/11/22 15:35:23 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=25 sets of demonstrations...


Bootstrapping set 1/25
Bootstrapping set 2/25


 15%|█▌        | 3/20 [01:19<07:28, 26.36s/it]


Bootstrapped 3 full traces after 3 examples for up to 1 rounds, amounting to 3 attempts.
Bootstrapping set 3/25


 10%|█         | 2/20 [01:07<09:38, 32.16s/it]<unknown>:5: SyntaxWarning: invalid escape sequence '\m'
<unknown>:9: SyntaxWarning: invalid escape sequence '\m'
<unknown>:13: SyntaxWarning: invalid escape sequence '\m'
<unknown>:23: SyntaxWarning: invalid escape sequence '\m'
 15%|█▌        | 3/20 [01:32<08:46, 30.96s/it]


...

<unknown>:11: SyntaxWarning: invalid escape sequence '\m'


Average Metric: 65.63 / 75 (87.5%):  92%|█████████▎| 74/80 [00:48<00:05,  1.18it/s]

<unknown>:9: SyntaxWarning: invalid escape sequence '\m'


Average Metric: 70.15 / 80 (87.7%): 100%|██████████| 80/80 [01:11<00:00,  1.11it/s]

2024/11/22 16:46:16 INFO dspy.evaluate.evaluate: Average Metric: 70.14814814814815 / 80 (87.7%)
2024/11/22 16:46:16 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [87.96, 88.06, 89.17, 87.69]
2024/11/22 16:46:16 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 89.17
2024/11/22 16:46:16 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2024/11/22 16:46:16 INFO dspy.teleprompt.mipro_optimizer_v2: 

2024/11/22 16:46:16 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 89.17!


In [48]:

# Save optimize program for future use
zeroshot_optimized_program.save(f"medium_100trainset_v1")


In [49]:
evaluate(zeroshot_optimized_program, devset=devset, metric=metric)

  0%|          | 0/20 [00:00<?, ?it/s]

<unknown>:5: SyntaxWarning: invalid escape sequence '\m'


Average Metric: 12.22 / 14 (87.3%):  70%|███████   | 14/20 [00:47<00:12,  2.12s/it]

<unknown>:13: SyntaxWarning: invalid escape sequence '\m'


Average Metric: 13.15 / 15 (87.7%):  75%|███████▌  | 15/20 [00:50<00:11,  2.30s/it]

<unknown>:5: SyntaxWarning: invalid escape sequence '\!'
<unknown>:9: SyntaxWarning: invalid escape sequence '\!'
<unknown>:23: SyntaxWarning: invalid escape sequence '\!'


Average Metric: 15.81 / 20 (79.1%): 100%|██████████| 20/20 [01:11<00:00,  3.58s/it]

2024/11/22 16:54:52 INFO dspy.evaluate.evaluate: Average Metric: 15.814814814814815 / 20 (79.1%)


,context,reasoning,ontology,metric
0,"To some, stoichiometry is no more exciting than the law of conserv...",The text discusses the concept of stoichiometry and its importance...,"entities=[Entity(name='Stoichiometry', information='Stoichiometry ...",✔️ [0.889]
1,If a glass tube that is open at both ends stands upright in a cont...,The text describes a physical experiment involving a glass tube an...,"entities=[Entity(name='Glass_Tube', information='A glass tube that...",✔️ [0.889]
2,This is the Schrodinger equation of a free particle moving in one ...,The text describes a scenario involving a particle subjected to a ...,"entities=[Entity(name='Particle', information='A particle moving i...",✔️ [0.889]
3,It is interesting to note that the nitrogen atom also shows little...,The text discusses the stability of the nitrogen atom and its anio...,"entities=[Entity(name='Nitrogen_Atom(N)', information='The nitroge...",✔️ [0.926]
4,generally the central focus of a problem is the conversion of a ma...,The text discusses the process of converting mass in grams to mole...,"entities=[Entity(name='Mass', information='Mass is a measure of th...",✔️ [0.926]
5,The three subatomic particles considered in this section are the o...,"The text discusses subatomic particles, specifically electrons, ne...","entities=[Entity(name='Electron', information='The electron is bel...",✔️ [0.704]
6,"As a result of experiments of the type just described, we find tha...",The text describes a phenomenon related to the photoelectric effec...,"entities=[Entity(name='StoppingVoltage(Vs)', information='The stop...",✔️ [0.741]
7,of blood. Estimates of BAC can be obtained from breath samples by ...,The text discusses the estimation of Blood Alcohol Concentration (...,"entities=[Entity(name='Blood Alcohol Concentration(BAC)', informat...",✔️ [0.889]
8,Electrostatic potential maps are a powerful way of displaying the ...,The text discusses the concept of electrostatic potential maps and...,"entities=[Entity(name='Electrostatic Potential Map', information='...",✔️ [0.889]
9,Our major undertaking in this section will be to describe the thre...,The text discusses the three-dimensional probability density distr...,"entities=[Entity(name='Hydrogen Atom', information='The hydrogen a...",✔️ [0.926]


79.07

In [139]:
zeroshot_optimized_program = ChemOntology()
zeroshot_optimized_program.load(f"medium_100trainset_v1")


In [143]:
tests = ["""
###### How electron donation and withdrawal change chemical shifts  \nWe can get an idea of the effect of electron distribution by looking at a series of benzene rings\nwith the same substituent in the 1 and 4 positions. This pattern makes all four hydrogens on\nthe ring identical. Here are a few compounds listed in order of chemical shift: largest shift\n(lowest fi eld; most deshielded) fi rst. Conjugation is shown by the usual curly arrows, and\ninductive effects by a straight arrow by the side of the group. Only one hydrogen atom and\none set of arrows are shown. 
""",
"""
\nConjugation, as discussed in\nChapter 7, is felt through π bonds,\nwhile inductive effects are the\nresult of electron withdrawal or\ndonation felt simply by polarization\nof the σ bonds of the molecule.\nSee p. 135.  \nthe effect of electron-withdrawing groups\nby conjugation  \nby inductive effects  \n**H**  \n**O**  \n**O**  \n**HO**  \n**N**  \nδH 8.48 δH 8.10 **C** δH 8.10 δH 8.07 δH 7.78  \n**N**  \n**O**  \n**O**  \n**OH**  \n**C**  \n**N**  \n**O**  \n**H**  \n**F** **F**  \n**F**  \nThe largest shifts come from groups that withdraw electrons by conjugation. 
""",
"""
Nitro is the\nmost powerful—this should not surprise you as we saw the same in non-aromatic compounds\nin both [13]C and [1]H NMR spectra. Then come the carbonyl and nitrile group followed by groups\nshowing simple inductive withdrawal. CF3 is an important example of this kind of group—\nthree fl uorine atoms combine to exert a powerful effect.  \n-----  \nIn the middle of our sequence, around the position of benzene itself at 7.27 ppm, come\nthe halogens, whose inductive electron withdrawal and lone pair donation are nearly\nbalanced.  \nbalance between withdrawal by inductive effect and donation of lone pairs by conjugation  \n**I** δH 7.40 **Br** δH 7.32 δH 7.27 **Cl** δH 7.24 **F** δH 7.00  \n**I**  \n**Br**  \n**Cl**  \n**F**  
""",
"""
\nAlkyl groups are weak inductive donators, but the groups which give the most shielding—\nperhaps surprisingly—are those containing the electronegative atoms O and N. Despite being\ninductively electron withdrawing (the C–O and C–N σ bonds are polarized with δ + C), on\nbalance conjugation of their lone pairs with the ring (as you saw on p. 278) makes them net\nelectron donors. They increase the shielding at the ring hydrogens. 
""",
"""
Amino groups are the best.\nNote that one nitrogen-based functional group (NO2) is the best electron withdrawer while\nanother (NH2) is the best electron donor.  \nthe effect of electron-donating groups  \nby inductive effect  \nbalance between withdrawal by inductive effect and donation\nof lone pairs by conjugation—electron donation wins  \n**H**  \nδH 7.03  \n**H**  \n**H**  \n**CH3**  \nδH 6.80 **O**  \n**H** **H**  \nδH 6.59 **N**  \n**H**  \nδH 6.35  \n**H**  \n**H**  \n**CH3**  \n**O**  \n**CH3**  \n**H**  \n**H**  \n**N**  \n**O**  \nδH 7.27  \n**H**  \n**H**  \nδH 7.27  \nδH 5.68  \n**H**  \n**H**  \nδH 5.68  \n**O**  \nδH 6.0  \n**H**  \n**H**  \nδH 7.0  \nδH 4.65  \n**H**  \n**H**  \nδH 6.35  \nAs far as the donors with lone pairs are concerned (the halogens plus O and N), two factors\nare important—the size of the lone pairs and the electronegativity of the element. 
""",
"""
If we look\nat the four halides at the top of this page the lone pairs are in 2p (F), 3p (Cl), 4p (Br), and 5p (I)\norbitals. In all cases the orbitals on the benzene ring are 2p so the fl uorine orbital is of the\nright size to interact well and the others too large. Even though fl uorine is the most electronegative, it is still the best donor. The others don’t pull so much electron density away, but\nthey can’t give so much back either.\nIf we compare the fi rst row of the p block elements—F, OH, and NH2—all have lone pairs\nin 2p orbitals so now electronegativity is the only variable. As you would expect, the most\nelectronegative element, F, is now the weakest donor.
"""
]



In [145]:
for test in tests:
    print(ontology_to_string(zeroshot_optimized_program(test).ontology))





Entities:
  - Name: Electron Donation
    Information: Electron donation refers to the process by which an electron-rich species donates electrons to another species, affecting chemical shifts.
  - Name: Electron Withdrawal
    Information: Electron withdrawal is the process by which an electron-deficient species attracts electrons from another species, affecting chemical shifts.
  - Name: Chemical Shift
    Information: Chemical shift is a change in the resonant frequency of a nucleus due to the electronic environment, often measured in NMR spectroscopy.
  - Name: Benzene Ring
    Information: A benzene ring is a cyclic hydrocarbon with alternating double bonds, often used as a model to study electron distribution effects.
  - Name: Substituent
    Information: A substituent is an atom or group of atoms that replaces hydrogen in a hydrocarbon, affecting the chemical properties of the molecule.
  - Name: Conjugation
    Information: Conjugation refers to the overlap of p-orbitals acros

In [153]:
for test in tests:
    print(metric(dspy.Prediction(context=test),zeroshot_optimized_program(test), verbose=True))



Entity Accuracy Score: 5
Data Property Correctness Score: 4
Object Property Completeness Score: 3
Ontology Structure Score: 4
Overall Score: 8

Total Score: 24
Percentage Score: 88.89%

Reason:
entity accuracy: 
data property correctness: The ontology lacks explicit capture of dependencies for the chemical shift value, which affects the assessment score.
object property completeness: The ontology lacks object property chains and inverse relationships, and the use of 'some' restrictions may not fully capture the specificity of interactions.
ontology structure: 
overall score: 


Entity Accuracy Score: 4
Data Property Correctness Score: 4
Object Property Completeness Score: 3
Ontology Structure Score: 4
Overall Score: 6

Total Score: 21
Percentage Score: 77.78%

Reason:
entity accuracy: The ontology does not fully capture an appropriate chemical classification hierarchy, which prevents it from achieving a perfect score.
data property correctness: The ontology lacks explicit mention of t

In [14]:
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
from dspy.predict import Retry

In [10]:
from autonogy_constructor.signatures import ExtractOntologyElements
class ChemOntologyWithAssertions(dspy.Module):
    def __init__(self):
        super().__init__()

        self.extractor = dspy.ChainOfThought(ExtractOntologyElements)
        self.assessor = Assessment(assertions=True)
    
    def forward(self, context):
        pred = self.extractor(text=context)
        ontology = pred.ontology
        assertions, score_info = self.assessor(assessed_text=context, assessment_ontology=ontology)
        qualified = True
        suggestion = ""
        # print(qualified,suggestion)
        for key, value in assertions.items():
            # print(key)
            if not value[0]:
                qualified = False
                suggestion += key + ": " + value[1] + "\n"
                # print(key)
        # print(qualified,suggestion)
        dspy.Suggest(qualified,suggestion,target_module=self.extractor)
        return dspy.Prediction(context=context, ontology=ontology, score_info=score_info)


In [15]:
from dspy.evaluate import Evaluate

evaluate = Evaluate(metric=metric, devset=devset_no_ontology, num_threads=8, display_progress=True, display_table=10)
chemonto_with_assertions = assert_transform_module(ChemOntologyWithAssertions().map_named_predictors(Retry), backtrack_handler) 

In [16]:
evaluate(chemonto_with_assertions)

2024/11/23 20:24:43 INFO dspy.primitives.assertions: SuggestionFailed: entity: The ontology may not be effectively applied across different chemical subdomains, as it is specifically tailored to the interaction of alpha particles with foil, which is a very specific scenario. To improve, consider designing entities that can be generalized or adapted for broader applications in other chemical contexts.
ontology_structure: - The ontology does not fully span all levels of chemical granularity, particularly the statistical nature of the interactions.
- The semantic clarity could be improved by providing more precise definitions of behaviors.
- Cross-references between the statistical data and behaviors are lacking, reducing the meaningfulness and accuracy of the ontology.

To improve, the ontology should include more detailed entities or properties that reflect the statistical nature of the interactions and provide clearer definitions and cross-references to enhance clarity and accuracy.
ov

[5, 3, 5, 2, 17]
reason_list
assertions
  0%|          | 0/3 [00:00<?, ?it/s]

2024/11/23 20:25:20 INFO dspy.primitives.assertions: SuggestionFailed: entity: The ontology is limited in its applicability across different chemical subdomains. To improve, it could include more general entities or properties that describe particle interactions with various materials, making it more versatile for different chemical contexts.
object_property: The relationship restriction could be improved by specifying conditions or probabilities for each type of interaction, rather than using a general "some" restriction. This would enhance the precision of the ontology in reflecting the experimental observations.
ontology_structure: - Entities span all levels of chemical granularity mentioned in the text: The ontology does not fully span all levels of chemical granularity, as it lacks specific details about the foil material and the experimental context.
- Cross-references between concepts are meaningful and accurate: The ontology could improve by explicitly linking the behaviors to 

[5, 3, 4, 3, 21]
reason_list
assertions


2024/11/23 20:25:33 INFO dspy.primitives.assertions: SuggestionFailed: entity: The ontology is limited in its applicability across different chemical subdomains. To improve, it could include more general entities or properties that describe particle interactions with various materials, making it more versatile for different chemical contexts.
object_property: The relationship restriction could be improved by specifying conditions or probabilities for each type of interaction, rather than using a general "some" restriction. This would enhance the precision of the ontology in reflecting the experimental observations.
ontology_structure: - Entities span all levels of chemical granularity mentioned in the text: The ontology does not fully span all levels of chemical granularity, as it lacks specific details about the foil material and the experimental context.
- Cross-references between concepts are meaningful and accurate: The ontology could improve by explicitly linking the behaviors to 

...

2024/11/23 20:32:06 INFO dspy.primitives.assertions: SuggestionFailed: object_property: The ontology does not capture all key chemical interactions, such as the correlation of electron configurations with physical and chemical properties. Additionally, the "correlates_with" property is incomplete, and the use of "some" in "is_distributed_among" may not fully capture the complexity of electron distribution. Improvements should include adding missing interactions and refining relationship restrictions.
ontology_structure: The ontology loses a point because the properties and relationships do not form a fully coherent chemical knowledge graph. The "correlates_with" property is incomplete, lacking a proper range entity, which affects the coherence and completeness of the ontology. To improve, ensure all properties have complete and accurate domain and range specifications, and verify that all relationships are meaningful and accurately reflect the chemical concepts they are intended to rep

[6, 3, 3, 3, 20]
reason_list
assertions
[6, 3, 3, 3, 20]
0.8366666666666667
Average Metric: 2.34 / 3 (78.1%): 100%|██████████| 3/3 [07:23<00:00, 147.73s/it]

2024/11/23 20:32:06 INFO dspy.evaluate.evaluate: Average Metric: 2.341666666666667 / 3 (78.1%)


,example_context,pred_context,ontology,score_info,metric
0,· The majority of $\alpha$ particles penetrated the foil undeflect...,· The majority of $\alpha$ particles penetrated the foil undeflect...,"entities=[Entity(name='Alpha_Particles', information='Alpha partic...",Entity Score: 0.8333333333333334 Data Property Score: 1.0 Object P...,✔️ [0.852]
1,34. Explain why these reactions cannot occur as written. (a) (b) ...,34. Explain why these reactions cannot occur as written. (a) (b) ...,"entities=[Entity(name='Iron(III) ion (Fe^{3+}(aq))', information='...",Entity Score: 0.6666666666666666 Data Property Score: 1.0 Object P...,✔️ [0.653]
2,The electron configuration of an atom is a designation of how elec...,The electron configuration of an atom is a designation of how elec...,"entities=[Entity(name='Atom', information='An atom is the basic un...",Entity Score: 1.0 Data Property Score: 1.0 Object Property Score: ...,✔️ [0.837]


78.06

In [21]:
from dspy.teleprompt import MIPROv2, BootstrapFewShot

teleprompter = BootstrapFewShot(
    metric=metric,
    teacher_settings=dict(lm=teacher),
    max_bootstrapped_demos=8,
    max_labeled_demos=8,
    max_rounds=10,
)

zeroshot_optimized_program = teleprompter.compile(
    student = chemonto_with_assertions,
    teacher = chemonto_with_assertions,
    trainset=trainset_no_ontology,
    # requires_permission_to_run=False,
    # max_bootstrapped_demos=0, # ZERO FEW-SHOT EXAMPLES
    # max_labeled_demos=0, # ZERO FEW-SHOT EXAMPLES
)


  0%|          | 0/10 [00:00<?, ?it/s]2024/11/23 20:34:53 INFO dspy.primitives.assertions: SuggestionFailed: entity: The entities may not be applied effectively across different chemical subdomains without modification. To improve transferability, consider including more general definitions or categorizations that could apply to a variety of chemical reactions, regardless of the specific reactants or products involved.
object_property: The ontology captures the interaction and products correctly, but the Domain for the 'produces' property should specify which entity produces the product. Additionally, domain specification for 'produces' could instead focus on both reactants as input entities, emphasizing their collective role in the reaction rather than attempting to apply without clarity to individual alignments.
ontology_structure: - Entities do not span all levels of chemical granularity mentioned; "mole" is not explicitly an entity.
- Definitions and usage of entities could be more

[5, 3, 4, 3, 15]
reason_list
assertions


2024/11/23 20:36:06 INFO dspy.primitives.assertions: SuggestionFailed: object_property: While the ontology correctly captures the chemical interactions and principles, the use of 'some' as the only restriction might suggest combinations beyond the specified stoichiometry of the chemical reaction, which could be made more precise with 'only' restrictions considering the defined context of fixed reactant amounts.
overall_content: 1. **Extraction Accuracy (5/6):** High accuracy but missed emphasizing the unit of enthalpy change per mole as a critical detail.
2. **Professional Validity (5/6):** Aligns well with scientific principles overall, requiring clearer separation of ΔrH as per mole.
3. **Comprehensiveness (3/6):** Captures most elements, but deeper explication of ΔrH versus total enthalpy change could improve completeness.
4. **Knowledge Independence (2/6):** Good independence but some aspects require text-based context for clear interpretation.



[6, 3, 4, 4, 15]
reason_list
assertions


2024/11/23 20:39:12 INFO dspy.primitives.assertions: SuggestionFailed: object_property: The object properties do capture some key chemical interactions, but there are points of enhancement, such as integrating the detailed mechanics of energy involvement and refining relationship restrictions. The current setup meets domain and range necessities and holds validity, yet could benefit from broader complexity acknowledgment in 'only' applications or chemical reaction intricacies involving energy changes more thoroughly.
overall_content: There are slight inaccuracies in the representation of "enthalpy_of_reaction_per_mole." The ontology could also enhance professional validity and comprehensiveness by specifying more details, especially conditions affecting the enthalpy change, and elaborating thoroughly on "enthalpy_of_reaction_per_mole." Improving clarity in these areas would facilitate understanding and align better with expert-level chemistry.

 10%|█         | 1/10 [05:08<46:19, 308.8

...

[4, 3, 4, 3, 15]
reason_list
assertions


2024/11/23 20:59:59 INFO dspy.primitives.assertions: SuggestionFailed: object_property: The object's properties capture several, but not all, key chemical interactions described in the text (e.g., gradation of electron density). It's recommended to add specific object properties capturing subtle interactions like partial electron transfer, as mentioned in the text for a more comprehensive representation.



[6, 3, 4, 4, 20]
reason_list
assertions


2024/11/23 21:01:11 INFO dspy.primitives.assertions: SuggestionFailed: object_property: To improve the ontology, the following enhancements should be considered:

- Introduce additional object properties or refine existing ones to include more detailed chemical interactions, such as partial charge interactions for polar compounds like HCl. 

- Consider simplifying domain and range classifications where they are not adding substantial value or necessary precision, focusing instead on flexibility to accommodate various chemical characteristics described in the text.

- Incorporate enhancements based on the details found in the electrostatic potential maps and how they relate to electron charge distribution, to more accurately model the molecular behavior described in the assessed text.
overall_content: Extraction Accuracy: Minor non-critical errors such as misalignment in property attribution for molecules.
Professional Validity: Mostly scientifically valid, but nuances in representation

[6, 3, 3, 4, 16]
reason_list
assertions
[6, 3, 3, 4, 16]
0.8533333333333333
Bootstrapped 8 full traces after 8 examples for up to 10 rounds, amounting to 10 attempts.


In [22]:
evaluate(zeroshot_optimized_program)

  0%|          | 0/3 [00:00<?, ?it/s][6, 3, 5, 5, 21]
reason_list
assertions
[5, 2, 4, 4, 18]
0.8099999999999999
Average Metric: 0.81 / 1 (81.0%):  33%|███▎      | 1/3 [01:11<02:22, 71.11s/it]

2024/11/23 21:03:48 INFO dspy.primitives.assertions: SuggestionFailed: object_property: To improve the ontology, the following enhancements should be considered:

- Introduce additional object properties to capture specific chemical interactions, such as electron transfer or changes in oxidation states, which are crucial for understanding why the reactions cannot occur as written.

- Simplify or remove unnecessary domain and range specifications for `participates_in` to focus on the essential chemical interactions.

- Ensure that the `reaction_viability` property is supported by detailed explanations of thermodynamic, kinetic, or stoichiometric constraints that prevent the reactions from occurring.
ontology_structure: The ontology captures the entities and their states but fails to form a coherent knowledge graph that explains the reactions' non-viability. It lacks semantic clarity and meaningful cross-references to chemical principles that would elucidate why the reactions cannot occu

[6, 3, 3, 2, 19]
reason_list
assertions


2024/11/23 21:05:05 INFO dspy.primitives.assertions: SuggestionFailed: object_property: The ontology needs to be expanded to include specific object properties that capture the chemical interactions and principles relevant to the reactions. This includes detailing redox processes, electron transfers, and stoichiometric considerations that explain why the reactions cannot occur as written. The current structure is too generic to provide meaningful insights into the chemical processes described in the text.
ontology_structure: The ontology captures the entities and their states well but lacks detailed object properties and cross-references that explain the chemical interactions and constraints. To improve, the ontology should include properties that describe the chemical reactions and the reasons they cannot occur as written, such as charge balance or redox potential issues. This would enhance the coherence and meaningfulness of the chemical knowledge graph.
overall_content: The ontology

[6, 3, 3, 3, 19]
reason_list
assertions


...

2024/11/23 21:07:59 INFO dspy.primitives.assertions: SuggestionFailed: data_property: The ontology correctly identifies potential data properties related to electron configurations, but the text does not provide explicit values for these properties. Therefore, while the ontology is accurate in its structure, it cannot fulfill the criterion of having explicitly stated values from the text.



[6, 2, 5, 5, 20]
reason_list
assertions
[5, 2, 3, 3, 18]
0.72
Average Metric: 2.13 / 3 (71.2%): 100%|██████████| 3/3 [06:18<00:00, 126.14s/it]

2024/11/23 21:08:32 INFO dspy.evaluate.evaluate: Average Metric: 2.135 / 3 (71.2%)


,example_context,pred_context,ontology,score_info,metric
0,· The majority of $\alpha$ particles penetrated the foil undeflect...,· The majority of $\alpha$ particles penetrated the foil undeflect...,"entities=[Entity(name='Alpha Particles (α Particles)', information...",Entity Score: 1.0 Data Property Score: 1.0 Object Property Score: ...,✔️ [0.810]
1,34. Explain why these reactions cannot occur as written. (a) (b) ...,34. Explain why these reactions cannot occur as written. (a) (b) ...,"entities=[Entity(name='Iron(III) Ion (Fe³⁺)', information='A catio...",Entity Score: 1.0 Data Property Score: 1.0 Object Property Score: ...,✔️ [0.605]
2,The electron configuration of an atom is a designation of how elec...,The electron configuration of an atom is a designation of how elec...,"entities=[Entity(name='Electron Configuration', information=""The a...",Entity Score: 1.0 Data Property Score: 0.6666666666666666 Object P...,✔️ [0.720]


71.17

In [6]:

chemonto_with_assertions = assert_transform_module(ChemOntologyWithAssertions().map_named_predictors(Retry), backtrack_handler) 
print(ontology_to_string(chemonto_with_assertions(test_context).ontology))
print(chemonto_with_assertions(test_context).score_info)

2024/11/23 19:53:32 ERROR dspy.primitives.assertions: AssertionError: entity: - The ontology does not fully capture concepts across all levels of abstraction, such as the balance between electron withdrawal and donation, and the specific effects of lone pair size and electronegativity.
- The design does not explicitly support future expansion or integration with other ontologies, as it lacks a broader framework for incorporating additional chemical concepts or subdomains.
object_property: The ontology does not explicitly capture the balance between inductive withdrawal and lone pair donation for halogens, which is a key point in the text. Additionally, the specificity of domain and range specifications could be improved by including more specific entities for certain interactions, such as specifying halogens individually.
ontology_structure: The ontology lacks granularity in representing all substituents and specific electron effects mentioned in the text. It does not fully capture the

True 
entity
entity
data_property
object_property
object_property
ontology_structure
ontology_structure
overall_content
overall_content
False entity: - The ontology does not fully capture concepts across all levels of abstraction, such as the balance between electron withdrawal and donation, and the specific effects of lone pair size and electronegativity.
- The design does not explicitly support future expansion or integration with other ontologies, as it lacks a broader framework for incorporating additional chemical concepts or subdomains.
object_property: The ontology does not explicitly capture the balance between inductive withdrawal and lone pair donation for halogens, which is a key point in the text. Additionally, the specificity of domain and range specifications could be improved by including more specific entities for certain interactions, such as specifying halogens individually.
ontology_structure: The ontology lacks granularity in representing all substituents and specif

DSPyAssertionError: entity: - The ontology does not fully capture concepts across all levels of abstraction, such as the balance between electron withdrawal and donation, and the specific effects of lone pair size and electronegativity.
- The design does not explicitly support future expansion or integration with other ontologies, as it lacks a broader framework for incorporating additional chemical concepts or subdomains.
object_property: The ontology does not explicitly capture the balance between inductive withdrawal and lone pair donation for halogens, which is a key point in the text. Additionally, the specificity of domain and range specifications could be improved by including more specific entities for certain interactions, such as specifying halogens individually.
ontology_structure: The ontology lacks granularity in representing all substituents and specific electron effects mentioned in the text. It does not fully capture the complexity of interactions, such as the balance b

In [ ]:
print(ontology_to_string(ChemOntology()(test_context).ontology))

In [7]:
print(metric(dspy.Prediction(context=test_context),ChemOntology()(test_context), verbose=True))



Entity Score: 0.8333333333333334
Hierachy Score: 1.0
Disjointness Score: 1.0
Data Property Score: 1.0
Object Property Score: 0.8
Ontology Structure Score: 0.75
Overall Content Score: 0.75

Total Score: 45/54
Percentage Score: 87.62%

Reason:
Entity: The ontology does not fully capture the nuanced balance between electron withdrawal and donation effects, particularly for halogens, as discussed in the text. To improve, the ontology could include more detailed properties or entities that reflect this balance, enhancing its applicability across different chemical subdomains.
Hierachy: 
Disjointness: 
Data Property: 
Object Property: The ontology does not capture all key chemical interactions, as it omits object properties for other electron-withdrawing groups like carbonyl, nitrile, and CF3. To improve, additional object properties should be added to represent these interactions, ensuring a more comprehensive representation of chemical shifts.
Ontology Structure: The ontology does not ful